In [1]:
import os
import pandas as pd
from PIL import Image
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import transforms, models
from torch.utils.data import Dataset, DataLoader, random_split
import numpy as np
from sklearn.metrics import accuracy_score, classification_report
import matplotlib.pyplot as plt

In [2]:
BATCH_SIZE = 128
EPOCHS = 10
LEARNING_RATE = 1e-4
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
SELECTED_ATTRIBUTES = ['Bald', 'Bangs', 'Big_Lips', 'Big_Nose', 'Black_Hair', 'Blond_Hair', 'Brown_Hair', 'Bushy_Eyebrows', 'Chubby',
       'Eyeglasses', 'Goatee', 'Gray_Hair', 'Heavy_Makeup', 'High_Cheekbones', 'Mustache', 'Narrow_Eyes', 'No_Beard',
       'Oval_Face', 'Pale_Skin', 'Pointy_Nose', 'Receding_Hairline',
       'Rosy_Cheeks', 'Sideburns', 'Straight_Hair', 'Wavy_Hair',
       'Wearing_Earrings', 'Wearing_Hat', 'Wearing_Lipstick',
       'Wearing_Necklace', 'Wearing_Necktie']
IMG_DIR = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\celeba\img_align_celeba"
ATTR_FILE = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\celeba\list_attr_celeba.txt"

In [ ]:
class CelebADataset(Dataset):
    def __init__(self, img_dir, attr_file, selected_attributes, transform=None):
        self.img_dir = img_dir
        self.transform = transform

        # load/preprocess attributes
        self.attributes = pd.read_csv(attr_file, delim_whitespace=True, skiprows=1)

        # fix filenames
        self.attributes.index = self.attributes.index.map(lambda x: f"{x.zfill(6)}.jpg" if not x.endswith(".jpg") else x)
        
        # make binary
        self.attributes = self.attributes[selected_attributes].replace(-1, 0)

    def __len__(self):
        return len(self.attributes)

    def __getitem__(self, idx):
        # image name and path
        img_name = self.attributes.index[idx]
        img_path = os.path.join(self.img_dir, img_name)

        # open image
        try:
            image = Image.open(img_path).convert("RGB")
        except FileNotFoundError as e:
            print(f"Error: {e} - Image {img_name} not found.")
            raise e

        # Get labels as a tensor
        labels = torch.tensor(self.attributes.iloc[idx].values, dtype=torch.float32)

        # apply transformations
        if self.transform:
            image = self.transform(image)

        return image, labels

In [ ]:
train_transforms = transforms.Compose([
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomRotation(degrees=15),
    transforms.RandomResizedCrop((224, 224), scale=(0.8, 1.0)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])

val_transforms = transforms.Compose([
    transforms.CenterCrop((160, 160)),
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
])
#augmentatiuons

In [5]:
dataset = CelebADataset(img_dir=IMG_DIR, attr_file=ATTR_FILE, selected_attributes=SELECTED_ATTRIBUTES, transform=train_transforms)


C:\Users\shrit\AppData\Local\Temp\ipykernel_16688\3189453638.py:7: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  self.attributes = pd.read_csv(attr_file, delim_whitespace=True, skiprows=1)


In [ ]:
train_size = int(0.8 * len(dataset))
val_size = len(dataset) - train_size
train_dataset, val_dataset = random_split(dataset, [train_size, val_size])

In [ ]:
val_dataset.dataset.transform = val_transforms

In [8]:
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True, num_workers=0)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE, shuffle=False, num_workers=0)

In [ ]:
class FaceAttributeModel(nn.Module):
    def __init__(self, num_features):
        super(FaceAttributeModel, self).__init__()
        self.base_model = models.mobilenet_v3_large(weights=models.MobileNet_V3_Large_Weights.IMAGENET1K_V1) # imagenet weights, mobilenetV3
        in_features = self.base_model.classifier[0].in_features
        self.base_model.classifier = nn.Sequential()  #  original classifier

        self.fc1 = nn.Linear(in_features, 512)
        self.dropout = nn.Dropout(0.3)
        self.fc2 = nn.Linear(512, num_features)

    def forward(self, x):
        x = self.base_model(x)
        x = self.fc1(x)
        x = torch.relu(x)
        x = self.dropout(x)
        x = self.fc2(x)
        return torch.sigmoid(x)
#actual model

In [ ]:
from tqdm import tqdm

def train_one_epoch(model, train_loader, criterion, optimizer, device):
    model.train()
    running_loss = 0.0
    correct, total = 0, 0  #  accuracy counters

    for images, labels in tqdm(train_loader, desc="Training", leave=False):
        images, labels = images.to(device), labels.to(device)

        # Forward
        outputs = model(images)
        loss = criterion(outputs, labels)

        # Back prop
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()

        running_loss += loss.item()

        # Accuracy 
        predicted = (outputs > 0.5).float()
        correct += (predicted == labels).sum().item()
        total += labels.numel()

    accuracy = correct / total  
    print(f"Training Accuracy: {accuracy:.4f}")  #  accuracy
    return running_loss / len(train_loader)

def validate_one_epoch(model, val_loader, criterion, device):
    model.eval()
    running_loss = 0.0
    correct, total = 0, 0  #  accuracy counters

    with torch.no_grad(): # no back prop
        for images, labels in tqdm(val_loader, desc="Validating", leave=False):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)
            running_loss += loss.item()

            predicted = (outputs > 0.5).float()
            correct += (predicted == labels).sum().item()
            total += labels.numel()

    accuracy = correct / total 
    print(f"Validation Accuracy: {accuracy:.4f}")  # accuracy
    return running_loss / len(val_loader)

In [11]:
model = FaceAttributeModel(num_features=len(SELECTED_ATTRIBUTES)).to(DEVICE)
criterion = nn.BCELoss()  # Binary cross-entropy loss
optimizer = optim.Adam(model.parameters(), lr=LEARNING_RATE)


In [ ]:
model = model.to("cuda")  # GPU
print(next(model.parameters()).device) 

cuda:0


In [ ]:
save_dir = "C:\\Users\\shrit\\Desktop\\Ml_Projects\\Ml_Projects\\pytorch_results_weights"
os.makedirs(save_dir, exist_ok=True)
best_model_path = os.path.join(save_dir, "best_model_state_dict.pth")  #  path for state_dict

best_val_loss = float('inf')
for epoch in range(EPOCHS):
    print(f"Epoch [{epoch+1}/{EPOCHS}]")

    # Training and validation
    train_loss = train_one_epoch(model, train_loader, criterion, optimizer, DEVICE)
    val_loss = validate_one_epoch(model, val_loader, criterion, DEVICE)

    # Save best model
    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), best_model_path)  # Save  state_dict
        print(f"Best model state_dict saved with validation loss: {best_val_loss:.4f}")

    print(f"Train Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}") 

Epoch [1/10]


Training Accuracy: 0.8975


Validation Accuracy: 0.9138
Best model state_dict saved with validation loss: 0.1966
Train Loss: 0.2383, Val Loss: 0.1966
Epoch [2/10]


Training Accuracy: 0.9141


Validation Accuracy: 0.9176
Best model state_dict saved with validation loss: 0.1873
Train Loss: 0.1957, Val Loss: 0.1873
Epoch [3/10]


...

Training Accuracy: 0.9201


Validation Accuracy: 0.9194
Best model state_dict saved with validation loss: 0.1829
Train Loss: 0.1821, Val Loss: 0.1829
Epoch [5/10]


Training Accuracy: 0.9221


Validation Accuracy: 0.9199
Best model state_dict saved with validation loss: 0.1823
Train Loss: 0.1774, Val Loss: 0.1823
Epoch [6/10]


KeyboardInterrupt: 

In [ ]:
all_labels = []
all_preds = []

model.eval()

with torch.no_grad():
    for images, labels in val_loader:
        images, labels = images.to(DEVICE), labels.to(DEVICE)

        outputs = model(images)
        predictions = (outputs > 0.5).float() 

        all_preds.append(predictions.cpu().numpy())
        all_labels.append(labels.cpu().numpy())

all_preds = np.concatenate(all_preds, axis=0)
all_labels = np.concatenate(all_labels, axis=0)

accuracy = accuracy_score(all_labels.flatten(), all_preds.flatten())
print(f"Validation Accuracy: {accuracy:.4f}")

print(classification_report(all_labels, all_preds, target_names=SELECTED_ATTRIBUTES))

Validation Accuracy: 0.9198
                   precision    recall  f1-score   support

             Bald       0.80      0.75      0.77       930
            Bangs       0.88      0.85      0.86      6140
         Big_Lips       0.60      0.37      0.46      9834
         Big_Nose       0.72      0.59      0.65      9421
       Black_Hair       0.81      0.80      0.81      9740
       Blond_Hair       0.86      0.86      0.86      6008
       Brown_Hair       0.74      0.65      0.69      8354
   Bushy_Eyebrows       0.80      0.59      0.68      5678
           Chubby       0.68      0.52      0.59      2306
       Eyeglasses       0.98      0.97      0.97      2681
           Goatee       0.78      0.76      0.77      2518
        Gray_Hair       0.81      0.72      0.76      1750
     Heavy_Makeup       0.90      0.90      0.90     15781
  High_Cheekbones       0.89      0.85      0.87     18555
         Mustache       0.66      0.54      0.59      1668
      Narrow_Eyes       0.6

c:\Users\shrit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\shrit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in samples with no true labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
c:\Users\shrit\AppData\Local\Programs\Python\Python312\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in samples with no true nor predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mod

In [ ]:
from torchvision import transforms
model.eval() 

preprocess = transforms.Compose([
    transforms.Resize((224, 224)), 
    transforms.ToTensor(),  
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  
])

image_path = r"C:\Users\shrit\Desktop\Ml_Projects\Ml_Projects\Data\celeba\img_align_celeba\000001.jpg"  
raw_image = Image.open(image_path).convert('RGB')  
image = preprocess(raw_image) 

image = image.unsqueeze(0).to(DEVICE)  

with torch.no_grad():
    outputs = model(image)  

probabilities = torch.sigmoid(outputs).cpu().numpy().flatten()  

feature_vector = probabilities

print("Feature Vector (Probabilities):")
print(feature_vector)

Feature Vector (Probabilities):
[0.50000036 0.5000213  0.54518944 0.5009237  0.5004308  0.5054338
 0.50880396 0.50006986 0.5000099  0.50004214 0.5000024  0.5000016
 0.5109977  0.52519935 0.5000051  0.5157874  0.730968   0.50299853
 0.5021717  0.53430116 0.5002037  0.50001615 0.500003   0.7090483
 0.5002853  0.5486195  0.534381   0.57198733 0.51722044 0.50000536]
